In [1]:
# load libraries
%matplotlib inline
import pandas as pd
import numpy as np
import time
import os
import re
from string import ascii_letters
import nltk 
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize 

In [2]:
ignore = ["around", "well", "often","including", "could", "much",
             "see", "also", "use","eg", "first", "across",
             "within", "among", "class","instructor", "consult", "syllabus",
             "graduate", "students", "taking","version", "criteria", "offered",
             "concurrently", "expectations", "differ","evaluation", "details", "meets",
             "when", "specific", "next","steps", "goals", "focuses",
             "required", "draws", "range","area", "deeply", "exploring",
             "focusing", "aimed", "introduces","includes", "examples", "issues",
             "person", "credit", "term","description", "subject", "listing",
             "oversubscribed", "priority", "given","course", "student", "addresses",
             "key", "questions", "day","class", "participate", "undergraduates",
             "prepare", "presents", "problems","setting", "complete", "additional",
             "assignments", "work", "must","understand", "involved", "concepts",
             "discusses", "solving", "exercises","weekly", "limited", "identify",
             "topics", "include", "using","provides", "instruction", "identifying",
             "lectures", "studies", "effective","skills", "capabilities", "mit",
             "approach", "take", "intended","variety", "uses", "successful",
             "solutions", "beyond", "resources", "emphasis", "problem", "covers",
              "building", "basic", "focused", "faced", "various",
              "new", "opportunities", "examines", "introduction", "impact", "domains",
              "used", "high", "demonstrates", "emphasizes", "process", "examines",
              "new", "basic", "principles", "may", "readings", "practice", "enrollment",
              "experience", "explores", "develop", "covers", "understanding", "various",
              "role"]


In [3]:
def word_extraction(sentence):
    lower = sentence.lower() #lowercase
    no_digs = ''.join([i for i in lower if not i.isdigit()]) #get rid of digits
    word_tokens = word_tokenize(no_digs) #tokenize
    filtered_sentence = [w for w in word_tokens if not w in stop_words] #get rid of stop words
    cleaned_text = [w for w in filtered_sentence if w not in ignore] #get rid of meaningless course words
    return cleaned_text


In [4]:
#read in the practice CSV as a Pandas DataFrame
df = pd.read_csv("draft1.csv", header=0)
#show the first five lines to verify upload
df.head()

,practice,Course_Num,Course_Name,CourseDescription,Year_Taken,First_Name,Last_name,Mat_Year,Gender,"Background (Business, Design, Engineering)",...,ExpectationsClearSD,LearningObjectivesMetMean,LearningObjectivesMetMedian,LearningObjectivesMetSD,AssignmentsContributedMean,AssignmentsContributedMedian,AssignmentsContributedSD,GradingFairMean,GradingFairMedian,GradingFairSD
0,1,15.232,Bus Models in Frontier Mkts,"Examines how new approaches to operations, rev...",2019,Aaron,Stinnett,2018,M,Business,...,1.738191,-1.017473,-0.052129,1.497737,-0.926698,-1.070141,1.306693,0.206078,0.395280,0.247746
1,1,15.518,Taxes and Business Strategy,Provides conceptual framework for thinking abo...,2020,Aaron,Stinnett,2018,M,Business,...,-0.974411,1.143502,1.044330,-0.707001,1.128727,0.836859,-0.439839,1.054206,0.904898,-0.937880
2,1,15.570,Digital Marketing,Analysis of the underlying economics of inform...,2018,Aaron,Stinnett,2018,M,Business,...,-0.550567,0.507921,-0.052129,-0.351398,0.357943,0.836859,0.052773,0.084916,-0.114337,0.001673
3,1,15.761,Intro Operations Management,"Imparts concepts, techniques, and tools to des...",2020,Aaron,Stinnett,2018,M,Business,...,-0.211492,0.380805,-0.052129,0.336100,0.486407,0.836859,-0.439839,0.690722,0.904898,-0.669437
4,1,15.915,Lab for Sustainable Business,"Develops a pragmatic, action-oriented approach...",2019,Aaron,Stinnett,2018,M,Business,...,1.314347,-1.398821,-1.148587,1.758512,-1.954410,-2.023641,1.776913,-1.490179,-1.133573,1.097819


In [5]:
#Initialize the output dataframe with a dummy row
init_data = [['asdfjkl']]
unique_df = pd.DataFrame(init_data, columns = ['Desc'])
#Verify the new dataframe looks as intended
unique_df.head()

,Desc
0,asdfjkl


In [6]:
desc_list = []
for i in range(len(df.index)):
    desc_list.append(df["CourseDescription"][i]) #build a list of all course descriptions
    
desc_set = set(desc_list)
unique_desc = list(desc_set) #get rid of the duplicates
len(unique_desc)

146

In [7]:
#Initialize the output dataframe with a dummy row
init_data = [['asdfjkl',0]]
wordcount_df = pd.DataFrame(init_data, columns = ['Word','Count'])
#Verify the new dataframe looks as intended
wordcount_df.head()

,Word,Count
0,asdfjkl,0


In [8]:
#Output dataframe index tracker
k=0

for i in range(len(unique_desc)):
    test_bag = unique_desc[i]
    
    #get rid of html ascii characters
    test_bag = test_bag.replace('&nbsp;',' ')
    test_bag = test_bag.replace('&amp;',' and ')
    test_bag = test_bag.replace('&quot;','')
    
    #get rid of literal chracters
    test_bag = test_bag.replace("'",'')
    test_bag = test_bag.replace(".",'')
    test_bag = test_bag.replace(",",'')
    test_bag = test_bag.replace("(",'')
    test_bag = test_bag.replace(")",'')
    test_bag = test_bag.replace("-",'')
    test_bag = test_bag.replace("!",'')
    test_bag = test_bag.replace(";",'')
    test_bag = test_bag.replace(":",'')
    test_bag = test_bag.replace("/",'')
    test_bag = test_bag.replace("?",'')
    test_bag = test_bag.replace("$",'')
    #clean up any double spaces
    test_bag = test_bag.replace("  ",' ')
    #remove ownership
    test_bag = test_bag.replace("'s",'')
    #some plurals by hand
    test_bag = test_bag.replace("networks",'network')
    test_bag = test_bag.replace("systems",'system')
    test_bag = test_bag.replace("cases",'case')
    test_bag = test_bag.replace("products",'product')
    test_bag = test_bag.replace("technologies",'technology')
    test_bag = test_bag.replace("projects",'project')
    test_bag = test_bag.replace("modeling",'models')
    test_bag = test_bag.replace("analyze",'analysis')
    test_bag = test_bag.replace("analytics",'analysis')
       
    #extract relevant words
    words = word_extraction(test_bag)
        
    #Iterate through all of the words in the cleaned bag of words
    for word in words:

        #Iterate through the output df and check for repeats
        repeat_flag = False
        for j in wordcount_df.index:
            #If you find a repeat, increment the count by one but don't do anything else
            if wordcount_df.at[j,'Word'] == word:
                wordcount_df.at[j,'Count'] += 1
                repeat_flag = True
                break;

        #If you don't find a repeat, add it to the output Df
        if repeat_flag == False:
            temp_data = [[word, 1]]
            temp_df = pd.DataFrame(temp_data, columns = ['Word', 'Count'])
            wordcount_df.loc[k] = temp_df.loc[0]
            #Move to the next row in the output DF
            k += 1

wordcount_df

,Word,Count
0,seeking,2
1,actionoriented,3
2,innovation,18
3,ecosystem,3
4,silicon,1
...,...,...
2199,impacts,1
2200,delays,1
2201,entrant,1
2202,airlines,1


In [ ]:
bag_o_words = []
for i in wordcount_df.index:
    #Constrain the number of columns - more is more detail per course, but also more sparse overall
    if wordcount_df["Count"][i] > 10:  #IMPORTANT VARIABLE - lower the cutoff to increase the number of columns
        bag_o_words.append(wordcount_df["Word"][i])
len(bag_o_words)  #check the number of columns we're adding

In [ ]:
#create the new "word" columns in the dataframe
for word in bag_o_words:
    df[word] = pd.Series(0,index=df.index)
df.head() #check it

In [ ]:
for i in df.index:
    desc = df.at[i,"CourseDescription"] #pull in the course description
    for word in bag_o_words:
        if word in desc: #count the number of times the word appeats in the desc and set the appropriate value
            df.at[i,word] = 1 #1 for logistic, desc.count(word) for linear
df

In [ ]:
#Save it as a CSV to the local directory
df.to_csv(r'bagofwords_smallset.csv', index = False)